# Simulation of multi-state models

In [2]:
# libraries
library(mstate)
library(flexsurv)
library(data.table)
set.seed(234)

Loading required package: survival



In [3]:
# read data
data(prothr)
dat = data.table(prothr)
head(dat)

id,from,to,trans,Tstart,Tstop,status,treat
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2,1,3,0,151,0,Placebo
1,2,3,4,0,151,1,Placebo
2,2,1,3,0,251,1,Placebo
2,2,3,4,0,251,0,Placebo
2,1,2,1,251,434,1,Placebo
2,1,3,2,251,434,0,Placebo


In [4]:
table(dat$trans)
dat[, years := Tstop - Tstart][years == 0, years := 0.001][, years := years / 365.25]
dat[, trans := as.factor(trans)]
tmat = attr(prothr, "trans")


  1   2   3   4 
532 532 544 544 

## Flexsurv

- Does this take into account uncertainty?
- No, we have to use bootstrap
- The bootstrap function might limit us

```
summfn <- function(x, ...){
    sim <-  sim.fmsm(x, ...) 
 ##  [[ compute the summary you want here. ]] 
}
mod <- flexsurvreg(...)

bootci.fmsm(mod, fn=summfn, ) 
```

In [6]:
tmat

,Normal,Low,Death
Normal,NA,1,2
Low,3,NA,4
Death,NA,NA,NA


In [7]:
# models 

# parametric models
crexp = flexsurvreg(Surv(years, status) ~ trans  + treat, data = dat, dist = "exp") 
crwei = flexsurvreg(Surv(years, status) ~ 1 + treat, anc = list(shape = ~ trans), data = dat, dist = "weibull")

In [8]:
crwei

Call:
flexsurvreg(formula = Surv(years, status) ~ 1 + treat, anc = list(shape = ~trans), 
    data = dat, dist = "weibull")

Estimates: 
                 data mean  est      L95%     U95%     se       exp(est)
shape                 NA     0.7393   0.6721   0.8132   0.0360       NA 
scale                 NA     5.1336   4.4882   5.8718   0.3519       NA 
treatPrednisone   0.5028     0.1081  -0.0680   0.2843   0.0899   1.1142 
shape(trans2)     0.2472     0.2956   0.1411   0.4501   0.0788   1.3440 
shape(trans3)     0.2528    -0.1045  -0.2329   0.0240   0.0655   0.9008 
shape(trans4)     0.2528    -0.2584  -0.3946  -0.1221   0.0695   0.7723 
                 L95%     U95%   
shape                 NA       NA
scale                 NA       NA
treatPrednisone   0.9342   1.3288
shape(trans2)     1.1515   1.5685
shape(trans3)     0.7922   1.0243
shape(trans4)     0.6740   0.8850

N = 2152,  Events: 880,  Censored: 1272
Total time at risk: 3555.401
Log-likelihood = -1991.459, df = 6
AIC = 399

In [9]:
AIC(crexp) - AIC(crwei)

[1] -211.8338

In [10]:
# list of independent models
crwei.list <- list()
trans = order(unique(dat$trans))
for (i in trans) {
    crwei.list[[i]] =  flexsurvreg(Surv(years, status) ~ 1 + treat, subset=(trans==i), data = dat,  dist = "weibull")
}

## Simulate trajectories

- The function `sim.fsms` simulates one or more trajectories for the same set of covariates (nrow == 1)
- To simulate different individual with different characteristics, we need to loop over those characteristics
- For parameter uncertainty, we can use bootstrap

In [125]:
simulateMS = function(model, newdata, time, idvar = "id", tmat = NULL, iterations = 1,
    checkDuplicates = TRUE) {
    
    # some initial check
    if (checkDuplicates && anyDuplicated(newdata[, get(idvar)]) > 0) {
        stop("There are duplicated ids in newdata")
    }
                              
    nsimulations = nrow(newdata)
    output = list()
    print(paste0(":::: Number of individuals to simulate over ", time, " units of time: ", nsimulations))
    for (i in 1:nsimulations) {
        if (i %% 100 == 0) {print(paste0(":::: Simulating individual number ", i))}
        
        temp = sim.fmsm(model, M = iterations, t = time, newdata = newdata[i, ], trans = tmat)
        
        a = data.table(temp$st)[, iter := 1:.N]; setnames(a, gsub("V", "s", names(a)))
        b = data.table(temp$t)[, iter := 1:.N]; setnames(b, gsub("V", "t", names(b)))
        temp = merge(a, b, on = "iter")
        
        sid = newdata[i, get(idvar)]
        temp = melt(temp, id.vars = "iter", measure = patterns("^s", "^t"), 
            value.name = c("state", "time"))[, 
                `:=` (variable = NULL, id = sid)]

        temp = unique(temp, by = c("iter", "state" , "time"))
        setorder(temp, id, iter)                                                                          
        output[[i]] = temp
        
    }
    return(rbindlist(output))
}

In [126]:
newdata = unique(dat, by = "id")[, .(id, treat)]
test = simulateMS(crwei.list, newdata = newdata, time = 60, tmat = tmat, iterations = 10)

[1] ":::: Number of individuals to simulate over 60 units of time: 488"
[1] ":::: Simulating individual number 100"
[1] ":::: Simulating individual number 200"
[1] ":::: Simulating individual number 300"
[1] ":::: Simulating individual number 400"


### Resampling

- Create sampling
- Run models
- Simulation transitions